# Random Forest using kaggle dataset


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'test.csv', 'gender_submission.csv']

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)



from matplotlib import pyplot as plt
import seaborn as sns
data= pd.read_csv("../input/train.csv")
data.head()

In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None

In [ ]:
#no of survived people in each class
survived_class = data[data['Survived']==1]['Pclass'].value_counts()
didntsurvive_class = data[data['Survived']==0]['Pclass'].value_counts()
df_class = pd.DataFrame([survived_class,didntsurvive_class])
df_class.index = ['Survived','Died']
display(df_class)
data['Died'] = 1 - data['Survived']
data.groupby('Pclass').agg('mean')[['Survived', 'Died']]

In [ ]:
#no of survived people in each sex
survived_sex = data[data['Survived']==1]['Sex'].value_counts()
didntsurvive_sex = data[data['Survived']==0]['Sex'].value_counts()
df_sex = pd.DataFrame([survived_sex,didntsurvive_sex])
df_sex.index = ['Survived','Died']

#print(data[['Sex','Survived']].groupby(['Sex'],as_index=False).mean())
data['Died'] = 1 - data['Survived']
data.groupby('Sex').agg('mean')[['Survived', 'Died']]

In [ ]:
#combining two features into Familysize
for lines in data:
    data['Familysize']=data['SibSp']+data['Parch']+ 1
data.groupby('Familysize').agg('mean')[['Survived', 'Died']]

Familysize	Survived	Died
1	0.303538	0.696462
2	0.552795	0.447205
3	0.578431	0.421569
4	0.724138	0.275862
5	0.200000	0.800000
6	0.136364	0.863636
7	0.333333	0.666667
8	0.000000	1.000000
11	0.000000	1.000000

In [ ]:
# merging train data and test data for future feature engineering
def get_combined_data():
    # reading train data
    train = pd.read_csv("../input/train.csv")
    
    # reading test data
    test = pd.read_csv("../input/test.csv")

    # extracting and then removing the targets from the training data 
    targets = train.Survived
    #train.drop(['Survived'], 1, inplace=True)
    


    # we'll also remove the PassengerID since this is not an informative feature
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop(['index'],1,inplace=True)
    return combined

In [ ]:
combined = get_combined_data()


In [ ]:
X = combined.copy()                    
y = data.Survived 

In [ ]:
#extracting title
got= (X.Name.str.split(',').str[1]).str.strip()
X.Name=pd.DataFrame(got).Name.str.split('.').str[0]


In [ ]:
#encoding sex
print ('Number of null values in Sex:', sum(X.Sex.isnull()))
X.Sex.replace({ 'male':1, 'female':2}, inplace=True)

In [ ]:
#treating missing values in age and encoding
print ('Number of null values in Age:', sum(X.Age.isnull()))
X['Age']=X['Age'].fillna(X['Age'].mean())
print ('Number of null values in Age:', sum(X.Age.isnull()))

In [ ]:
#treating missing values in embarked 
print ('Number of null values in Embarked:', sum(X.Embarked.isnull()))

# fill the two values with one of the options (S, C or Q)
for dat in X:
    X['Embarked']=X['Embarked'].fillna('S')

In [ ]:
#treating missing values in embarked 
print ('Number of null values in Embarked:', sum(X.Embarked.isnull()))

# fill the two values with one of the options (S, C or Q)
for dat in X:
    X['Embarked']=X['Embarked'].fillna('S')

In [ ]:
X.groupby('Cabin').agg('mean')[['Survived']]
Survived


Cabin Survived
0	  0.299854
1	  0.666667

In [ ]:
for lines in X:
    X['Familysize']=X['SibSp']+X['Parch']+ 1
#data.groupby('Familysize').agg('mean')[['Survived', 'Died']

In [ ]:
#survived rate based on Title    
X[['Name','Survived']].groupby(['Name'],as_index=False).agg('mean')

Name	Survived
0	Capt	0.000000
1	Col	0.500000
2	Don	0.000000
3	Dona	NaN
4	Dr	0.428571
5	Jonkheer	0.000000
6	Lady	1.000000
7	Major	0.500000
8	Master	0.575000
9	Miss	0.697802
10	Mlle	1.000000
11	Mme	1.000000
12	Mr	0.156673
13	Mrs	0.792000
14	Ms	1.000000
15	Rev	0.000000
16	Sir	1.000000
17	the Countess	1.000000

In [ ]:
#encoding embarked
X['Embarked']=X.Embarked.replace({'S':0,'C':1,'Q':2}).astype(int)

In [ ]:
#mapping name
X['Title']=X['Name'].replace('the Countess','Royal')
X['Title']=X.Title.replace(['Lady','Countess','Capt','Col', 'Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Royal')
X['Title']=X['Title'].replace('Mlle','Miss')
X['Title']=X['Title'].replace('Ms','Miss')
X['Title']=X['Title'].replace('Mme','Mrs')
X.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royal'], dtype=object)

In [ ]:
#encoding title
X.head(5)
#title_mapping={'Mr':1,'Mrs':2,'Master':3,'Miss':4,'Rare':5}
#X['Title']=X['Title'].replace(title_mapping,inplace=True)
X['Title']=X['Title'].replace({'Mr':1,'Mrs':2,'Master':3,'Miss':4,'Royal':5}).astype(int)

In [ ]:
#Mapping Fare
X.loc[X.Fare <= 7.91,'Fare']=0
X.loc[(X.Fare > 7.91) & (X.Fare <= 14.454),'Fare']=1
X.loc[(X.Fare > 14.454) & (X.Fare <= 31),'Fare']=2
X.loc[X.Fare > 31,'Fare']=3

In [ ]:
#Mapping Age

X.loc[X.Age <= 16,'Age']=0
X.loc[(X.Age > 16) & (X.Age <= 32),'Age']=1
X.loc[(X.Age > 32) & (X.Age <= 48),'Age']=2
X.loc[(X.Age > 48) & (X.Age <= 64),'Age']=3
X.loc[X.Fare > 64,'Age']=4

In [ ]:
X.drop(['Ticket','Name','SibSp','PassengerId','Survived'],axis=1,inplace=True)
X.head(5)

In [ ]:
train = X.iloc[:891]
test = X.iloc[891:]
targets = pd.read_csv('../input/train.csv', usecols=['Survived'])['Survived'].values
train.head(5)

In [ ]:
sns.set(color_codes=True) #background grid
#sns.pairplot(train) #relation betn each pair of columns.
colormap=plt.cm.viridis
plt.figure(figsize= (14,14))
plt.title('Correlation of features',y=1.03,size =20)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,square=True,cmap=colormap,linecolor='black',annot=True)

In [ ]:



from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [ ]:

# Limit depth of tree to 3 levels
from sklearn.tree import export_graphviz
#import pydot
rf_small = RandomForestClassifier(n_estimators=10, max_depth = 3)
clf =rf_small.fit(train, targets)
# Extract the small tree
tree_small = rf_small.estimators_[5]

In [ ]:
# Save the tree as a png image
#export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
#(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
#graph.write_png('small_tree.png');

In [ ]:
features = pd.DataFrame()
features['feature'] = train.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)
features.plot(kind='barh', figsize=(5, 5))

In [ ]:
xval = cross_val_score(clf, train, targets, cv = 5, scoring='accuracy')
score=np.mean(xval)
print ('CV score = {0}'.format(score))

CV score = 0.8126021575388496



test.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 891 to 1308
Data columns (total 9 columns):
Age           418 non-null float64
Cabin         418 non-null int64
Embarked      418 non-null int64
Fare          418 non-null float64
Parch         418 non-null int64
Pclass        418 non-null int64
Sex           418 non-null int64
Familysize    418 non-null int64
Title         418 non-null int64
dtypes: float64(2), int64(7)
memory usage: 29.5 KB


In [ ]:
# Make predictions and determine the error
predictions = clf.predict(test).astype(int)
df_output = pd.DataFrame()
aux = pd.read_csv("../input/test.csv")

df_output['PassengerId'] = aux['PassengerId']
df_output['Survived'] = predictions

df_output[['PassengerId','Survived']].to_csv("randomforest.csv", index=False)